## 1. Setup Mountainsort
Before this notebook, need to do 
- (1) Clusterless sorters Notebook 5 in the General Dataprocessing first. That notebook detects lick artifact time.
- (2) decodePrep in Notebook 6. This step unions artifact times detected from the left and right hesmisphere and propagate to every tetrode so that every tetrode has the same artifact times. In addition, this step intersects with the track time obtained from Statescript.)

In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import os
#import cupy as cp
import numpy as np
import datajoint as dj
import spyglass as nd
import pandas as pd
import matplotlib.pyplot as plt
import json
import multiprocessing

# ignore datajoint+jupyter async warnings
import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore', category=ResourceWarning)

from spyglass.common import (Session, IntervalList,LabMember, LabTeam, Raw, Session, Nwbfile,
                            Electrode,LFPBand,interval_list_intersect, RawPosition, TaskEpoch)
import spyglass.spikesorting as ss

In [43]:
from spyglass.spikesorting.v0 import (SortGroup, 
                                    SortInterval,
                                    SpikeSortingPreprocessingParameters,
                                    SpikeSortingRecording, 
                                    SpikeSorterParameters,
                                    SpikeSortingRecordingSelection,
                                    ArtifactDetectionParameters, ArtifactDetectionSelection,
                                    ArtifactRemovedIntervalList, ArtifactDetection,
                                      SpikeSortingSelection, SpikeSorting,
                                   CuratedSpikeSortingSelection,CuratedSpikeSorting,Curation)
from spyglass.spikesorting.v0.curation_figurl import CurationFigurl,CurationFigurlSelection
from spyglass.spikesorting.v0.spikesorting_curation import MetricParameters,MetricSelection,QualityMetrics
from spyglass.spikesorting.v0.spikesorting_curation import WaveformParameters,WaveformSelection,Waveforms
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename
from pprint import pprint

from spyglass.shijiegu.helpers import interval_union
from spyglass.shijiegu.Analysis_SGU import TrialChoice,RippleTimes,EpochPos
from spyglass.shijiegu.load import load_run_sessions
from spyglass.shijiegu.singleUnit import do_mountainSort
from spyglass.shijiegu.decodeHelpers import runSessionNames,sleepSessionNames

### 0. Double check parameter

In [44]:
sorter_params_name = "CA1_tet_Shijie_whiten"
sorter_params = (SpikeSorterParameters & {"sorter_params_name": sorter_params_name}).fetch1("sorter_params")
sorter_params

{'detect_sign': -1,
 'adjacency_radius': -1,
 'freq_min': 0,
 'freq_max': 0,
 'filter': False,
 'whiten': True,
 'num_workers': 4,
 'clip_size': 39,
 'detect_threshold': 3,
 'detect_interval': 10,
 'verbose': True}

### 1. specify data

In [45]:
nwb_copy_file_name = "eliot20221024_.nwb"

In [46]:
SortInterval & {'nwb_file_name': nwb_copy_file_name}

nwb_file_name name of the NWB file,sort_interval_name name for this interval,sort_interval 1D numpy array with start and end time for a single interval to be used for spike sorting
eliot20221024_.nwb,02_Seq2Session2,=BLOB=
eliot20221024_.nwb,04_Seq2Session3,=BLOB=
eliot20221024_.nwb,06_Seq2Session4,=BLOB=
eliot20221024_.nwb,08_Seq2Session5,=BLOB=
eliot20221024_.nwb,10_Seq2Session6,=BLOB=
eliot20221024_.nwb,spikesort_maze,=BLOB=


In [47]:
intervals, _ = runSessionNames(nwb_copy_file_name)
print(intervals)

['02_Seq2Session2', '04_Seq2Session3', '06_Seq2Session4', '08_Seq2Session5', '10_Seq2Session6']


In [48]:
intervals2, _ = sleepSessionNames(nwb_copy_file_name)
print(intervals2)

['01_Seq2Sleep1', '03_Seq2Sleep2', '05_Seq2Sleep3', '07_Seq2Sleep4', '09_Seq2Sleep5', '11_Seq2Sleep6']


### 2. Do sorting

In [49]:
tetrode_with_cell=np.unique((SpikeSortingRecordingSelection & {'nwb_file_name':nwb_copy_file_name}).fetch('sort_group_id'))

In [50]:
tetrode_with_cell

array([  0,   2,   4,   5,  13,  14,  16,  17,  20,  26,  27,  28,  29,
        30,  31, 100, 101])

In [ ]:
for session_name in intervals:
    do_mountainSort(nwb_copy_file_name,session_name)

/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/base.py:998: UserWarning: Versions are not the same. This might lead compatibility errors. Using spikeinterface==0.97.1 is recommended
  warnings.warn(
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/binaryrecordingextractor.py:77: UserWarning: `num_chan` is to be deprecated in version 0.100, please use `num_channels` instead
  warnings.warn("`num_chan` is to be deprecated in version 0.100, please use `num_channels` instead")
[09:32:49][INFO] Spyglass: Running spike sorting on {'nwb_file_name': 'eliot20221024_.nwb', 'sort_group_id': 0, 'sort_interval_name': '02_Seq2Session2', 'preproc_params_name': 'franklab_tetrode_hippocampus', 'team_name': 'Shijie Gu', 'sorter': 'mountainsort4', 'sorter_params_name': 'CA1_tet_Shijie_whiten', 'artifact_removed_interval_list_name': 'eliot20221024_.nwb_02_Seq2Session2_0_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifac

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Using temporary directory /stelmo/nwb/tmp/tmprs6_mgy9
Using 4 workers.
Using tempdir: /stelmo/nwb/tmp/tmprs6_mgy9/tmpq4xqrnwe
Num. workers = 4
Preparing /stelmo/nwb/tmp/tmprs6_mgy9/tmpq4xqrnwe/timeseries.hdf5...
Preparing neighborhood sorters (M=4, N=51820491)...
Neighboorhood of channel 2 has 4 channels.
Detecting events on channel 3 (phase1)...
Elapsed time for detect on neighborhood: 0:00:04.134750
Num events detected on channel 3 (phase1): 122436
Computing PCA features for channel 3 (phase1)...
Clustering for channel 3 (phase1)...
Found 8 clusters for channel 3 (phase1)...
Computing templates for channel 3 (phase1)...
Re-assigning events for channel 3 (phase1)...
Neighboorhood of channel 3 has 4 channels.
Detecting events on channel 4 (phase1)...
Elapsed time for detect on neighborhood: 0:00:03.738171
Num events detected on channel 4 (phase1): 83378
Computing PCA features for channel 4 (phase1)...
Clustering fo

[09:34:32][INFO] Spyglass: Saving sorting results...
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/basesorting.py:212: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/base.py:998: UserWarning: Versions are not the same. This might lead compatibility errors. Using spikeinterface==0.97.1 is recommended
  warnings.warn(
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/binaryrecordingextractor.py:77: UserWarning: `num_chan` is to be deprecated in version 0.100, please use `num_channels` instead
  warnings.warn("`num_chan` is to be deprecated in version 0.100, please use `num_channels` instead")
[09:34:37][INFO] Spyglass: Running spike sorting on {'nwb_file_name': 'eliot2022

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Using temporary directory /stelmo/nwb/tmp/tmp1y5egk77
Using 4 workers.
Using tempdir: /stelmo/nwb/tmp/tmp1y5egk77/tmpy_nx1k8q
Num. workers = 4
Preparing /stelmo/nwb/tmp/tmp1y5egk77/tmpy_nx1k8q/timeseries.hdf5...
Preparing neighborhood sorters (M=4, N=51820491)...
Neighboorhood of channel 2 has 4 channels.
Detecting events on channel 3 (phase1)...
Elapsed time for detect on neighborhood: 0:00:03.694345
Num events detected on channel 3 (phase1): 77551
Computing PCA features for channel 3 (phase1)...
Clustering for channel 3 (phase1)...
Found 4 clusters for channel 3 (phase1)...
Computing templates for channel 3 (phase1)...
Re-assigning events for channel 3 (phase1)...
Neighboorhood of channel 1 has 4 channels.
Detecting events on channel 2 (phase1)...
Elapsed time for detect on neighborhood: 0:00:04.425930
Num events detected on channel 2 (phase1): 83570
Computing PCA features for channel 2 (phase1)...
Clustering for

[09:36:13][INFO] Spyglass: Saving sorting results...
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/basesorting.py:212: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/base.py:998: UserWarning: Versions are not the same. This might lead compatibility errors. Using spikeinterface==0.97.1 is recommended
  warnings.warn(
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/binaryrecordingextractor.py:77: UserWarning: `num_chan` is to be deprecated in version 0.100, please use `num_channels` instead
  warnings.warn("`num_chan` is to be deprecated in version 0.100, please use `num_channels` instead")
[09:36:18][INFO] Spyglass: Running spike sorting on {'nwb_file_name': 'eliot2022

...

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Using temporary directory /stelmo/nwb/tmp/tmpe1h76xmy
Using 4 workers.
Using tempdir: /stelmo/nwb/tmp/tmpe1h76xmy/tmp3clq1vcg
Num. workers = 4
Preparing /stelmo/nwb/tmp/tmpe1h76xmy/tmp3clq1vcg/timeseries.hdf5...
Preparing neighborhood sorters (M=4, N=50884418)...
Neighboorhood of channel 0 has 4 channels.
Detecting events on channel 1 (phase1)...
Elapsed time for detect on neighborhood: 0:00:03.416472
Num events detected on channel 1 (phase1): 106510
Computing PCA features for channel 1 (phase1)...
Clustering for channel 1 (phase1)...
Found 4 clusters for channel 1 (phase1)...
Computing templates for channel 1 (phase1)...
Re-assigning events for channel 1 (phase1)...
Re-assigning 520 events from 1 to 3 with dt=0 (k=2)
Re-assigning 81 events from 1 to 3 with dt=-1 (k=3)
Re-assigning 16 events from 1 to 3 with dt=-2 (k=4)
Neighboorhood of channel 2 has 4 channels.
Detecting events on channel 3 (phase1)...
Elapsed tim

[10:55:38][INFO] Spyglass: Saving sorting results...
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/basesorting.py:212: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/base.py:998: UserWarning: Versions are not the same. This might lead compatibility errors. Using spikeinterface==0.97.1 is recommended
  warnings.warn(
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/binaryrecordingextractor.py:77: UserWarning: `num_chan` is to be deprecated in version 0.100, please use `num_channels` instead
  warnings.warn("`num_chan` is to be deprecated in version 0.100, please use `num_channels` instead")
[10:55:43][INFO] Spyglass: Running spike sorting on {'nwb_file_name': 'eliot2022

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Using temporary directory /stelmo/nwb/tmp/tmp0hfrn_kr
Using 4 workers.
Using tempdir: /stelmo/nwb/tmp/tmp0hfrn_kr/tmpn0fd5kkp
Num. workers = 4
Preparing /stelmo/nwb/tmp/tmp0hfrn_kr/tmpn0fd5kkp/timeseries.hdf5...
Preparing neighborhood sorters (M=4, N=50884418)...
Neighboorhood of channel 1 has 4 channels.
Detecting events on channel 2 (phase1)...
Elapsed time for detect on neighborhood: 0:00:03.420311
Num events detected on channel 2 (phase1): 104539
Computing PCA features for channel 2 (phase1)...
Clustering for channel 2 (phase1)...
Found 3 clusters for channel 2 (phase1)...
Computing templates for channel 2 (phase1)...
Re-assigning events for channel 2 (phase1)...
Re-assigning 6419 events from 2 to 1 with dt=-1 (k=2)
Re-assigning 6 events from 2 to 1 with dt=-3 (k=3)
Neighboorhood of channel 0 has 4 channels.
Detecting events on channel 1 (phase1)...
Elapsed time for detect on neighborhood: 0:00:03.352310
Num ev

[10:57:10][INFO] Spyglass: Saving sorting results...
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/basesorting.py:212: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/base.py:998: UserWarning: Versions are not the same. This might lead compatibility errors. Using spikeinterface==0.97.1 is recommended
  warnings.warn(
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.9/site-packages/spikeinterface/core/binaryrecordingextractor.py:77: UserWarning: `num_chan` is to be deprecated in version 0.100, please use `num_channels` instead
  warnings.warn("`num_chan` is to be deprecated in version 0.100, please use `num_channels` instead")
[10:57:14][INFO] Spyglass: Running spike sorting on {'nwb_file_name': 'eliot2022

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Using temporary directory /stelmo/nwb/tmp/tmpwm7v8m8k
Using 4 workers.
Using tempdir: /stelmo/nwb/tmp/tmpwm7v8m8k/tmp3x9zb0qv
Num. workers = 4
Preparing /stelmo/nwb/tmp/tmpwm7v8m8k/tmp3x9zb0qv/timeseries.hdf5...
Preparing neighborhood sorters (M=4, N=50884418)...
Neighboorhood of channel 3 has 4 channels.
Detecting events on channel 4 (phase1)...
Elapsed time for detect on neighborhood: 0:00:03.576199
Num events detected on channel 4 (phase1): 115434
Computing PCA features for channel 4 (phase1)...
Clustering for channel 4 (phase1)...
Found 4 clusters for channel 4 (phase1)...
Computing templates for channel 4 (phase1)...
Re-assigning events for channel 4 (phase1)...
Re-assigning 26 events from 4 to 2 with dt=2 (k=3)
Neighboorhood of channel 0 has 4 channels.
Detecting events on channel 1 (phase1)...
Elapsed time for detect on neighborhood: 0:00:03.285177
Num events detected on channel 1 (phase1): 174097
Computing 

In [ ]:
for session_name in intervals2:
    do_mountainSort(nwb_copy_file_name,session_name)

In [ ]:
all_tet_list = np.unique((SpikeSorting & {"nwb_file_name": nwb_copy_file_name,"sorter":"mountainsort4"}).fetch("sort_group_id"))

In [ ]:
for tet in all_tet_list:
    key = {'nwb_file_name' : nwb_copy_file_name,
           "sort_group_id": tet,"sorter":"mountainsort4"}
    assert len(SpikeSortingRecordingSelection & key) == len(SpikeSortingRecording & key)
    
(SpikeSorting & {"nwb_file_name": nwb_copy_file_name,"sorter":"mountainsort4"})

In [19]:
(SpikeSorting & {"nwb_file_name": nwb_copy_file_name,"sorter":"mountainsort4"}).fetch("sort_interval_name")

array(['01_Seq2Sleep1', '02_Seq2Session1', '02_Seq2Session1',
       '03_Seq2Sleep2', '04_Seq2Session2', '04_Seq2Session2',
       '05_Seq2Sleep3', '06_Seq2Session3', '06_Seq2Session3',
       '07_Seq2Sleep4', '08_Seq2Session4', '10_Seq2Session5',
       '01_Seq2Sleep1', '02_Seq2Session1', '02_Seq2Session1',
       '03_Seq2Sleep2', '04_Seq2Session2', '04_Seq2Session2',
       '05_Seq2Sleep3', '06_Seq2Session3', '06_Seq2Session3',
       '07_Seq2Sleep4', '08_Seq2Session4', '10_Seq2Session5',
       '01_Seq2Sleep1', '02_Seq2Session1', '02_Seq2Session1',
       '03_Seq2Sleep2', '04_Seq2Session2', '04_Seq2Session2',
       '05_Seq2Sleep3', '06_Seq2Session3', '06_Seq2Session3',
       '07_Seq2Sleep4', '08_Seq2Session4', '10_Seq2Session5',
       '01_Seq2Sleep1', '02_Seq2Session1', '02_Seq2Session1',
       '03_Seq2Sleep2', '04_Seq2Session2', '04_Seq2Session2',
       '05_Seq2Sleep3', '06_Seq2Session3', '06_Seq2Session3',
       '07_Seq2Sleep4', '08_Seq2Session4', '10_Seq2Session5',
       '